<a href="https://colab.research.google.com/github/chuahwb/FNB-Imagery-AI-Tool/blob/main/notebooks/mllm_image_evaluation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# -*- coding: utf-8 -*-
"""
IPython Notebook for Phase 2: Evaluating Multimodal LLMs for F&B Image Recreation

This notebook connects to OpenRouter, processes local images, sends them with
prompts to selected multimodal LLMs, retrieves structured descriptions
using the 'instructor' library, tracks token usage, and estimates costs.
"""

# @title Setup: Install Libraries and Import Modules
# Install necessary libraries

"\nIPython Notebook for Phase 2: Evaluating Multimodal LLMs for F&B Image Recreation\n\nThis notebook connects to OpenRouter, processes local images, sends them with\nprompts to selected multimodal LLMs, retrieves structured descriptions\nusing the 'instructor' library, tracks token usage, and estimates costs.\n"

In [1]:
!pip install instructor openai python-dotenv pillow pandas tqdm Jinja2 openpyxl xlsxwriter -q # Added Jinja2 for HTML templating

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 169.4/169.4 kB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 345.6/345.6 kB 14.7 MB/s eta 0:00:00


In [2]:
import os
import base64
import instructor
from openai import OpenAI
from pydantic import BaseModel, Field, field_validator, PrivateAttr
from PIL import Image
from io import BytesIO
from typing import List, Optional, Tuple, Dict, Any
import pandas as pd
from tqdm.notebook import tqdm
from dotenv import load_dotenv, find_dotenv
import time
import datetime
import re
import traceback # For detailed error logging
from jinja2 import Environment, FileSystemLoader, select_autoescape # For HTML report
import html # For escaping text in HTML

In [3]:
import os
from google.colab import drive

def load_images_from_drive(dataset_path):
  """Loads images from Google Drive and returns a list of tuples.

  Args:
    dataset_path: The path to the dataset folder in Google Drive.

  Returns:
    A list of tuples, where each tuple contains the image ID, path, and category.
  """

  drive.mount('/content/drive')
  images_data = []
  category_counts = {}

  for category in os.listdir(dataset_path):
    category_path = os.path.join(dataset_path, category)
    if os.path.isdir(category_path):
      for image_file in os.listdir(category_path):
        if image_file.lower().endswith(('.png', '.jpg', '.jpeg')):
          image_id = os.path.splitext(image_file)[0]  # Use filename as ID
          image_path = os.path.join(category_path, image_file)
          images_data.append((image_id, image_path, category))
          category_counts[category] = category_counts.get(category, 0) + 1

  total_images = len(images_data)
  print(f"Total images loaded: {total_images}")
  print("Images loaded per category:")
  for category, count in category_counts.items():
    print(f"- {category}: {count}")

  return images_data

# Set the path to your dataset folder in Google Drive
dataset_path = '/content/drive/MyDrive/AI Imagery Marketing Tool/Colab Notebook/dataset_image'

# Load the images
IMAGES_TO_PROCESS = load_images_from_drive(dataset_path)

# Now IMAGES_TO_PROCESS contains your list of tuples
# You can use it in your existing code

Mounted at /content/drive
Total images loaded: 24
Images loaded per category:
- Product Shot: 13
- Menu Displays: 9
- Location Ambience Shots: 1
- Event Promotions: 1


In [18]:
# @title Configure API Key and OpenRouter Client

# --- IMPORTANT ---
# Set your OpenRouter API key.
# Option 1: Create a .env file in the same directory as this notebook
#           with the line: OPENROUTER_API_KEY="your-key-here"
# Option 2: Set it as an environment variable in your system.
# Option 3: Replace os.getenv("OPENROUTER_API_KEY") below with your actual key string
#           (less secure, not recommended for shared notebooks).
dotenv_path = "/content/drive/MyDrive/AI Imagery Marketing Tool/Colab Notebook/colab_secrets/.env"

if os.path.exists(dotenv_path):
    load_dotenv(dotenv_path=dotenv_path)
    print(f"Loaded .env file from path: {dotenv_path}")
else:
    print(f"Error: .env file not found at path: {dotenv_path}")

OPENROUTER_API_KEY = os.getenv("OPENROUTER_API_KEY")
GEMINI_API_KEY = os.getenv("GEMINI_API_KEY_1")

if not OPENROUTER_API_KEY:
    print("⚠️ OpenRouter API Key not found.")
    # OPENROUTER_API_KEY = input("Enter your OpenRouter API Key: ")
if not GEMINI_API_KEY:
    print("⚠️ Gemini API Key not found.")
    # GEMINI_API_KEY = input("Enter your Gemini API Key: ")

# Select LLM service provider
LLM_SERVICE_PROVIDER = "OpenRouter" # or "Gemini" or "openai" or "OpenRouter"
if LLM_SERVICE_PROVIDER == "OpenRouter":
  CLIENT_BASE_URL = "https://openrouter.ai/api/v1"
  CLIENT_API_KEY = OPENROUTER_API_KEY
elif LLM_SERVICE_PROVIDER == "Gemini":
  CLIENT_BASE_URL = "https://generativelanguage.googleapis.com/v1beta/openai/"
  CLIENT_API_KEY = GEMINI_API_KEY


# Configure the Instructor client to use OpenRouter
# Patch the OpenAI client to add structured response capabilities
# Store the original unpatched client for accessing raw response data if needed
# Note: Instructor v1+ modifies the client in place. We access usage from the returned pydantic model's _raw_response attribute.
client = instructor.patch(
    OpenAI(
        base_url=CLIENT_BASE_URL,
        api_key=CLIENT_API_KEY,
        default_headers={ # Optional, but good practice for OpenRouter
            "HTTP-Referer": "http://localhost:8888", # Replace with your app URL if deployed
            "X-Title": "F&B Image Eval", # Replace with your app name
        },
        timeout=600 # Increase timeout for potentially long image processing
    ),
    mode=instructor.Mode.MD_JSON # Use Markdown JSON mode for better compatibility
)

print(f"✅ OpenAI client patched with Instructor and configured for {LLM_SERVICE_PROVIDER}.")

Loaded .env file from path: /content/drive/MyDrive/AI Imagery Marketing Tool/Colab Notebook/colab_secrets/.env
✅ OpenAI client patched with Instructor and configured for OpenRouter.


In [19]:
# @title Define Pydantic Model for Structured Description
# This model mirrors the 8 points requested in the prompts

class FnbImageDescription(BaseModel):
    """Structured description of an F&B social media image."""
    primary_subject: str = Field(..., description="Detailed description of the main food, drink, person, or element, including ingredients, preparation, presentation, and actions.")
    composition_framing: str = Field(..., description="Description of layout (e.g., centered, rule of thirds), camera angle (e.g., eye-level, overhead), and framing (e.g., close-up, medium shot).")
    background_environment: str = Field(..., description="Details of the setting, including prominent foreground elements, background elements, surfaces, other objects, and depth of field.")
    lighting_color: str = Field(..., description="Description of light source, style (e.g., natural, studio), direction, shadows, highlights, dominant colors, and temperature.")
    texture_materials: str = Field(..., description="Specific textures visible (e.g., glossy sauce, crispy batter, smooth ceramic, condensation).")
    text_branding: str = Field(..., description="Description of the style, placement, and purpose of visible text and branding elements (e.g., font style, color, prominence, logo description). Avoids exact transcription unless critical for brand name.")
    mood_atmosphere: str = Field(..., description="Overall feeling conveyed by the image (e.g., cozy, vibrant, elegant, casual).")
    overall_style: str = Field(..., description="Characterization of the overall image style (e.g., photorealistic, illustration) including technical details like estimated camera effects (DoF, lens type), lighting style (cinematic, studio), rendering techniques, or post-processing.")
    # Store raw response for usage data access using PrivateAttr for internal use
    # This avoids the Pydantic field naming conflict.
    _raw_response: Optional[Any] = PrivateAttr(default=None)

    # Optional: Add a validator to ensure fields are not empty
    @field_validator('*', mode='before')
    def check_not_empty(cls, value):
        # This validator applies to the main fields, not the private attribute
        if isinstance(value, str) and not value.strip():
            return "(Not specified)" # Provide a default if empty
        return value

print("✅ Pydantic model 'FnbImageDescription' defined with updated field descriptions.")


✅ Pydantic model 'FnbImageDescription' defined with updated field descriptions.


In [20]:
# @title Define Image Handling Function

def encode_image_to_base64(image_path: str, max_size=(1024, 1024)) -> Optional[str]:
    """Loads an image, resizes if needed, and encodes it to base64."""
    try:
        with Image.open(image_path) as img:
            # Convert image to RGB if it's not (e.g., RGBA, P)
            if img.mode != 'RGB':
                img = img.convert('RGB')

            # Optional: Resize image to prevent exceeding token limits
            # Uncomment the line below if images are very large
            # print(f"    Original size: {img.size}")
            # img.thumbnail(max_size, Image.Resampling.LANCZOS)
            # print(f"    Resized to: {img.size}")

            buffered = BytesIO()
            img.save(buffered, format="JPEG", quality=85) # Save as JPEG with quality setting
            img_str = base64.b64encode(buffered.getvalue()).decode("utf-8")
            # print(f"    Encoded Base64 length: {len(img_str)}") # For debugging size
            return img_str
    except FileNotFoundError:
        print(f"❌ Error: Image file not found at {image_path}")
        return None
    except Exception as e:
        print(f"❌ Error encoding image {image_path}: {e}")
        return None

print("✅ Image encoding function 'encode_image_to_base64' defined.")


✅ Image encoding function 'encode_image_to_base64' defined.


In [21]:
# @title Define Prompt Construction Function

# Baseline Prompt (as defined previously)
BASELINE_PROMPT = """
Analyze the provided F&B image in meticulous detail. Generate a comprehensive description suitable for recreating this exact image using a text-to-image AI. Describe the following elements:

1.  **Primary Subject(s):** Identify and describe the main food, drink, person, or element. Include details like ingredients, preparation style (e.g., grilled, fried, steamed), presentation, specific actions (e.g., pouring, eating).
2.  **Composition & Framing:** Describe the layout (e.g., centered, rule of thirds, asymmetrical), camera angle (e.g., eye-level, overhead shot, low angle, Dutch tilt), and framing (e.g., extreme close-up, close-up, medium shot, full shot, wide shot).
3.  **Foreground, Background & Environment:** Detail the setting, specifically describing prominent **foreground elements**, background elements, surfaces (foreground and background), other objects present, and depth of field (e.g., sharp foreground with blurred background, deep focus).
4.  **Lighting & Color:** Describe the light source and style (e.g., bright natural daylight from window, warm indoor ambient light, dramatic studio flash, soft diffused light), direction of light, presence and softness of shadows, highlights, dominant color palette, and overall color temperature (e.g., warm tones, cool tones, vibrant, muted).
5.  **Texture & Materials:** Mention specific textures visible (e.g., glossy sauce, crispy batter, fluffy bread, smooth ceramic plate, rough wooden board, condensation on glass, metallic sheen of cutlery).
6.  **Text & Branding:** Describe the style, placement, and apparent purpose of any visible text (e.g., title, logo, caption, menu item). Note font style, color, and prominence. Avoid exact transcription unless essential for branding (like a main brand name). Describe logos or key branding elements.
7.  **Mood & Atmosphere:** Describe the overall feeling conveyed by the image (e.g., cozy and intimate, bright and energetic, rustic and homely, elegant and sophisticated, casual and fun, busy and dynamic).
8.  **Overall Style:** Characterize the overall image style (e.g., photorealistic, illustration, graphic design). Include technical details where possible, such as estimated camera/lens effects (e.g., shallow depth of field, wide-angle look), specific lighting style (e.g., cinematic, studio, natural low light), rendering techniques (e.g., cel-shaded, painterly), or post-processing hints (e.g., color grading, filters).
"""

# Category-Specific Emphasis (as defined previously)
CATEGORY_EMPHASIS = {
    "Product Shot": "Emphasis for Product Shot: Pay extra attention to the details of the food/drink item itself – texture, color accuracy, freshness indicators (e.g., steam, droplets), plating details, garnishes, and how the lighting highlights the product's appeal. Describe the dishware/glassware precisely.",
    "Lifestyle Shot": "Emphasis for Lifestyle Shot: Focus on the people involved – their expressions, actions, interactions with the product or each other, clothing style, and body language. Describe how the product is integrated into the scene and the overall narrative suggested (e.g., friends enjoying brunch, family dinner, solo coffee break).",
    "Menu Displays": "Emphasis for Menu Display: Prioritize accurate transcription of all visible text, including item names, descriptions, and prices. Describe the menu's layout, typography (font style, size, weight), color scheme, any graphical elements (lines, boxes, icons), and the material/context if it's a physical menu photo (e.g., chalkboard, printed paper, digital screen). Note the overall readability and design style.",
    "Promotional Graphics": "Emphasis for Promotional Graphic: Accurately transcribe all promotional text (offer details, dates, calls to action). Describe the graphic design elements used (e.g., background color/gradient, shapes, icons, font styles). If it combines photos and graphics, describe how they are integrated. Detail the overall visual hierarchy and intended message.",
    "Branding Elements": "Emphasis for Branding Element: Focus intensely on the specific branding element shown (e.g., logo, packaging detail, unique sign). Describe its colors, shapes, typography, and material precisely. Explain its context within the image and how it contributes to the overall brand identity.",
    "Location/Ambience Shots": "Emphasis for Location/Ambience: Describe the key features of the space – decor style (e.g., modern, rustic, industrial), furniture, lighting fixtures, color scheme, materials (wood, brick, metal), sense of space (cozy, spacious), cleanliness, and overall atmosphere it creates for a customer. Mention specific details like wall art, plants, table settings if visible.",
    "Event Promotions": "Emphasis for Event Promotion: Accurately transcribe all event details (name, date, time, location, description, contact info, price). Describe any specific imagery related to the event theme (e.g., musical instruments, wine bottles, specific food). Detail the overall design style of the flyer/poster/graphic and its call to action.",
    "Behind-the-Scenes (BTS)": "Emphasis for BTS: Describe the action taking place (e.g., cooking, plating, ingredient prep, staff interaction). Detail the environment (e.g., kitchen equipment, staff uniforms, raw ingredients) and the sense of activity or focus. Capture the candid, authentic feel typical of BTS shots.",
    "Default": "" # For categories not listed or if no emphasis is needed
}

def construct_prompt(category: Optional[str] = None, use_category_emphasis: bool = False) -> str:
    """Constructs the prompt, optionally adding category-specific emphasis."""
    prompt = BASELINE_PROMPT
    if use_category_emphasis and category:
        emphasis = CATEGORY_EMPHASIS.get(category, CATEGORY_EMPHASIS["Default"])
        if emphasis:
            prompt += "\n\n" + emphasis
    return prompt

print("✅ Prompt construction function 'construct_prompt' defined.")


✅ Prompt construction function 'construct_prompt' defined.


In [22]:
# @title Define Core Inference Function (with Token Usage)

def get_structured_description_with_usage(
    model_name: str,
    image_base64: str,
    prompt: str
) -> Tuple[Optional[FnbImageDescription], Optional[Dict[str, int]]]:
    """
    Sends image and prompt to a model via OpenRouter, gets a structured description,
    and extracts token usage information.
    Returns a tuple: (description_object, usage_dict)
    """
    usage_info = None
    description_obj = None
    try:
        print(f"   Querying {model_name}...")
        start_time = time.time()

        # Make the API call requesting the Pydantic model response
        description_obj = client.chat.completions.create(
            model=model_name,
            response_model=FnbImageDescription,
            max_retries=1, # Retry once on failure
            messages=[
                {
                    "role": "user",
                    "content": [
                        {"type": "text", "text": prompt},
                        {
                            "type": "image_url",
                            "image_url": {
                                "url": f"data:image/jpeg;base64,{image_base64}",
                                # OpenAI API suggests 'detail' param, but OpenRouter might not use it
                                # explicitly. High detail is usually default for base64.
                                # "detail": "high"
                            },
                        },
                    ],
                }
            ],
            max_tokens=2048, # Adjust as needed
            temperature=0.2, # Lower temperature for more deterministic descriptions
        )
        end_time = time.time()
        print(f"   ✅ Success for {model_name} in {end_time - start_time:.2f} seconds.")

        # --- Extract Token Usage ---
        # Access the private attribute _raw_response correctly
        raw_response = getattr(description_obj, '_raw_response', None)
        if raw_response and hasattr(raw_response, 'usage'):
             raw_usage = raw_response.usage
             if raw_usage:
                 usage_info = {
                     "prompt_tokens": raw_usage.prompt_tokens,
                     "completion_tokens": raw_usage.completion_tokens,
                     "total_tokens": raw_usage.total_tokens,
                 }
                 # print(f"      Usage: {usage_info}") # Uncomment for debugging
        else:
             # Fallback if usage is not directly on the response object's attribute
             # This might happen depending on instructor/openai library versions
             # Try accessing from the raw response dict if possible
             try:
                 if description_obj and hasattr(description_obj.model_extra, 'usage'):
                     raw_usage = description_obj.model_extra['usage']
                     usage_info = {
                         "prompt_tokens": raw_usage.prompt_tokens,
                         "completion_tokens": raw_usage.completion_tokens,
                         "total_tokens": raw_usage.total_tokens,
                     }
                     # print(f"      Usage (fallback): {usage_info}")
                 else:
                    print(f"   ⚠️ Usage information not found in response for {model_name}.")
                    usage_info = {"prompt_tokens": 0, "completion_tokens": 0, "total_tokens": 0}
             except Exception:
                 print(f"   ⚠️ Error accessing fallback usage info for {model_name}.")
                 usage_info = {"prompt_tokens": 0, "completion_tokens": 0, "total_tokens": 0}


    except Exception as e:
        print(f"   ❌ Error querying {model_name}:")
        # Print detailed traceback for debugging
        # traceback.print_exc()
        print(f"      {e}")
        # Ensure description_obj is None if error occurs before assignment
        description_obj = None
        usage_info = {"prompt_tokens": 0, "completion_tokens": 0, "total_tokens": 0} # Default on error

    return description_obj, usage_info

print("✅ Core inference function 'get_structured_description_with_usage' using response body usage check.")


✅ Core inference function 'get_structured_description_with_usage' using response body usage check.


In [23]:
# @title Model Configuration

# --- Configuration ---
# List of OpenRouter model identifiers to test (Update based on Step 1.2 and availability)
# Ensure these models support vision input on OpenRouter
MODELS_TO_TEST = [
    #"openai/gpt-4o-2024-11-20",
    #"openai/gpt-4.1-mini",
    "openai/o4-mini",
    #"openai/o3",
    # Anthropic - Claude
    #"anthropic/claude-3.7-sonnet",
    # Google - Gemini
    #"google/gemini-2.5-pro-preview-03-25",
    #"gemini-2.5-pro-exp-03-25",
    #"google/gemini-2.5-flash-preview",
    # Meta - Llama
    #"meta-llama/llama-4-scout:free",
    #"meta-llama/llama-4-maverick:free",
    #"meta-llama/llama-3.2-90b-vision-instruct",
    # XAI - Grok
    #"x-ai/grok-3-beta",
    # Qwen
    #qwen/qwen2.5-vl-32b-instruct",
]

# --- !!! IMPORTANT: Define Model Pricing (per Million Tokens) !!! ---
# GET THESE VALUES FROM https://openrouter.ai/models FOR ACCURACY
# Prices are in USD per 1 Million tokens (Input and Output)
MODEL_PRICING = {
    # Model Identifier: {"input_cost_per_M": X.XX, "output_cost_per_M": Y.YY}
    # OpenAI
    #"openai/gpt-4o-2024-11-20": {"input_cost_per_M": 2.50, "output_cost_per_M": 10.00},
    "openai/gpt-4.1-mini": {"input_cost_per_M": 0.40, "output_cost_per_M": 1.60},
    "openai/o4-mini": {"input_cost_per_M": 1.10, "output_cost_per_M": 4.40},
    #"openai/o3": {"input_cost_per_M": 10.00, "output_cost_per_M": 40.00},
    # Anthropic - Claude
    #"anthropic/claude-3.7-sonnet": {"input_cost_per_M": 3.00, "output_cost_per_M": 15.00},
    # Google - Gemini
    "google/gemini-2.5-pro-preview-03-25": {"input_cost_per_M": 1.25, "output_cost_per_M": 10.00},
    "gemini-2.5-pro-exp-03-25": {"input_cost_per_M": 0.00, "output_cost_per_M": 0.00},
    #"google/gemini-2.5-flash-preview": {"input_cost_per_M": 0.15, "output_cost_per_M": 0.6},
    # Meta - Llama
    #"meta-llama/llama-4-scout:free": {"input_cost_per_M": 0.0, "output_cost_per_M": 0.00},
    #"meta-llama/llama-4-maverick:free": {"input_cost_per_M": 0.0, "output_cost_per_M": 0.00},
    #"meta-llama/llama-3.2-90b-vision-instruct": {"input_cost_per_M": 0.8, "output_cost_per_M": 1.60},
    # XAI - Grok
    #"x-ai/grok-3-beta": {"input_cost_per_M": 3.0, "output_cost_per_M": 15.00},
    # Qwen
    "qwen/qwen2.5-vl-32b-instruct": {"input_cost_per_M": 0.90, "output_cost_per_M": 0.90},
}
print("⚠️ Ensure MODEL_PRICING dictionary is updated with current OpenRouter prices!")


# --- Input Data ---
# List of images to process. Each item is a tuple: (image_id, image_path, category)
# Replace with your actual image paths and categories from Step 1.1
# IMAGES_TO_PROCESS = [
#     ("prod_001", "path/to/your/product_shot_1.jpg", "Product Shot"),
#     ("life_001", "path/to/your/lifestyle_shot_1.png", "Lifestyle Shot"),
#     ("menu_001", "path/to/your/menu_display_1.jpg", "Menu Displays"),
#     ("promo_001", "path/to/your/promo_graphic_1.jpeg", "Promotional Graphics"),
#     # Add all other images from your dataset here...
# ]

⚠️ Ensure MODEL_PRICING dictionary is updated with current OpenRouter prices!


In [24]:
# @title Define Main Processing Workflow (with Cost Estimation)
# --- Workflow Execution ---

results_list = []

# Use tqdm for progress bar
for image_id, image_path, category in tqdm(IMAGES_TO_PROCESS, desc="Processing Images"):
    if image_id not in ["menu_0003"]:
      continue
    print(f"\nProcessing Image: {image_id} ({category}) - {image_path}")

    # 1. Encode Image
    image_base64 = encode_image_to_base64(image_path)
    if not image_base64:
        print(f"   Skipping image {image_id} due to encoding error.")
        continue

    # 2. Construct Prompt (Choose whether to use category emphasis)
    # Set use_category_emphasis=True to add specific instructions
    use_category_emphasis_flag = True # Or True
    prompt_text = construct_prompt(category, use_category_emphasis=use_category_emphasis_flag)

    # 3. Iterate through models
    for model_name in tqdm(MODELS_TO_TEST, desc=f"  Models for {image_id}", leave=False):
        description_obj, usage_info = get_structured_description_with_usage(model_name, image_base64, prompt_text)

        # --- Calculate Estimated Cost ---
        estimated_cost = 0.0
        prompt_tokens = 0
        completion_tokens = 0
        total_tokens = 0

        if usage_info:
            prompt_tokens = usage_info.get("prompt_tokens", 0)
            completion_tokens = usage_info.get("completion_tokens", 0)
            total_tokens = usage_info.get("total_tokens", 0)

            if model_name in MODEL_PRICING:
                pricing = MODEL_PRICING[model_name]
                input_cost = (prompt_tokens / 1_000_000) * pricing["input_cost_per_M"]
                output_cost = (completion_tokens / 1_000_000) * pricing["output_cost_per_M"]
                estimated_cost = input_cost + output_cost
            else:
                print(f"   ⚠️ Pricing not found for model {model_name}. Cost estimation skipped.")

        # Store results
        result_data = {
            "Image ID": image_id,
            "Image Path": image_path,
            "Category": category,
            "Model": model_name,
            "Prompt Type": "Category-Specific" if use_category_emphasis_flag and category else "Baseline",
            "Prompt Tokens": prompt_tokens,
            "Completion Tokens": completion_tokens,
            "Total Tokens": total_tokens,
            "Estimated Cost (USD)": round(estimated_cost, 6) # Round to 6 decimal places
        }

        if description_obj:
            # Add structured fields to the result dictionary
            # Use model_dump() which correctly handles Pydantic models without private attributes
            result_data.update(description_obj.model_dump())
            result_data["Status"] = "Success"
        else:
            # Add empty fields if the description failed
            # Iterate through the model's defined fields
            for field_name in FnbImageDescription.model_fields:
                 result_data[field_name] = "ERROR"
            result_data["Status"] = "Error"

        results_list.append(result_data)

print("\n✅ Workflow finished.")


Processing Images:   0%|          | 0/24 [00:00<?, ?it/s]


Processing Image: menu_0003 (Menu Displays) - /content/drive/MyDrive/AI Imagery Marketing Tool/Colab Notebook/dataset_image/Menu Displays/menu_0003.JPG


  Models for menu_0003:   0%|          | 0/1 [00:00<?, ?it/s]

   Querying qwen/qwen2.5-vl-32b-instruct...
   ✅ Success for qwen/qwen2.5-vl-32b-instruct in 16.38 seconds.

✅ Workflow finished.


In [25]:
# @title Display Results in a DataFrame

# Convert results to DataFrame early for easier processing
results_df = pd.DataFrame() # Initialize empty DataFrame
if results_list:
    results_df = pd.DataFrame(results_list)

    # Define column order for better readability
    display_column_order = [
        "Image ID", "Category", "Model", "Status", "Prompt Type",
        "Prompt Tokens", "Completion Tokens", "Total Tokens", "Estimated Cost (USD)",
        # Add the description fields
        "primary_subject", "composition_framing", "background_environment",
        "lighting_color", "texture_materials", "text_branding",
        "mood_atmosphere", "overall_style",
        "Image Path" # Include image path for reference
    ]
    # Ensure all expected columns exist before reordering
    results_df = results_df.reindex(columns=display_column_order, fill_value=None)


    # Set display options for better readability
    pd.set_option('display.max_rows', 50) # Adjust as needed
    pd.set_option('display.max_columns', None)
    pd.set_option('display.width', 1000) # Adjust for wider terminal/output
    pd.set_option('display.max_colwidth', 100) # Adjust width as needed

    print("\n--- Comparison Results (DataFrame) ---")
    # Display the DataFrame
    display(results_df)

    # --- Optional: Calculate and Display Summary Statistics ---
    print("\n--- Summary Statistics ---")
    # Total cost
    total_estimated_cost = results_df["Estimated Cost (USD)"].sum()
    print(f"Total Estimated Cost: ${total_estimated_cost:.6f}")

    # Average cost per model
    avg_cost_per_model = results_df.groupby("Model")["Estimated Cost (USD)"].mean()
    print("\nAverage Estimated Cost per Model:")
    display(avg_cost_per_model)

    # Average tokens per model
    avg_tokens_per_model = results_df.groupby("Model")[["Prompt Tokens", "Completion Tokens", "Total Tokens"]].mean()
    print("\nAverage Tokens per Model:")
    display(avg_tokens_per_model)


    # --- Optional: Save results to CSV ---
    # try:
    #     results_df.to_csv("fnb_llm_evaluation_results_with_cost.csv", index=False)
    #     print("\n✅ Results saved to fnb_llm_evaluation_results_with_cost.csv")
    # except Exception as e:
    #     print(f"\n❌ Error saving results to CSV: {e}")

else:
    print("\nNo results generated.")



--- Comparison Results (DataFrame) ---


,Image ID,Category,Model,Status,Prompt Type,Prompt Tokens,Completion Tokens,Total Tokens,Estimated Cost (USD),primary_subject,composition_framing,background_environment,lighting_color,texture_materials,text_branding,mood_atmosphere,overall_style,Image Path
0,menu_0003,Menu Displays,qwen/qwen2.5-vl-32b-instruct,Success,Category-Specific,3525,813,4338,0.003904,"The primary subjects are two sandwiches displayed prominently. The top sandwich is a hearty, mul...","The composition follows a symmetrical layout with the two sandwiches placed centrally, one above...","The background is a textured, beige-brown paper-like surface, giving the image a rustic and casu...","The lighting is warm and soft, likely simulating natural daylight. The light source appears to b...","The textures are varied and detailed. The top sandwich has a crispy, golden-brown bun with a sli...","The text is prominently displayed in bold, playful fonts. The main heading ""灵感简餐"" (Inspiration S...","The overall mood is cozy and inviting, with a casual and homely feel. The warm lighting, rustic ...",The overall style is photorealistic with a touch of graphic design elements. The image combines ...,/content/drive/MyDrive/AI Imagery Marketing Tool/Colab Notebook/dataset_image/Menu Displays/menu...



--- Summary Statistics ---
Total Estimated Cost: $0.003904

Average Estimated Cost per Model:


,Estimated Cost (USD)
Model,
qwen/qwen2.5-vl-32b-instruct,0.003904



Average Tokens per Model:


,Prompt Tokens,Completion Tokens,Total Tokens
Model,,,
qwen/qwen2.5-vl-32b-instruct,3525.0,813.0,4338.0


In [14]:
# @title Export Result to JSONL/CSV

# Example filtering (adjust as needed)
selected_model = "openai/o4-mini" # Or your best model

# Assuming you've added manual scores back to the df or have another way to filter
# For simplicity, let's just select successful runs from one model for now:
selected_descriptions_df = results_df[
    (results_df['Model'] == selected_model) &
    (results_df['Status'] == 'Success')
].copy() # Filter for the best model's successful runs

# Select relevant columns (don't need tokens/cost here)
columns_to_export = [
    "Image ID", "Image Path", "Category", "Model", # Keep original model info
    "primary_subject", "composition_framing", "background_environment",
    "lighting_color", "texture_materials", "text_branding",
    "mood_atmosphere", "overall_style"
]
export_df = selected_descriptions_df[columns_to_export]

# Export to CSV
# --- Prepare for Excel Export ---
now = datetime.datetime.now()
# Use underscores in timestamp for filename safety
formatted_date_time = now.strftime("%Y-%m-%d_%H-%M-%S")

export_filename = f"image_descriptions_{formatted_date_time}.csv"
export_df.to_csv(export_filename, index=False)
print(f"✅ Selected descriptions exported to {export_filename}")

# OR Export to JSON Lines (good for structured text)
export_filename_jsonl = f"image_descriptions_{formatted_date_time}.jsonl"
export_df.to_json(
    export_filename_jsonl,
    orient='records',      # Structure for JSON Lines
    lines=True,            # Ensure one JSON object per line
    force_ascii=False,     # *** THIS IS THE KEY CHANGE *** Allows UTF-8 characters
)

print(f"✅ Selected descriptions exported to {export_filename_jsonl}")

✅ Selected descriptions exported to image_descriptions_2025-04-23_21-35-07.csv
✅ Selected descriptions exported to image_descriptions_2025-04-23_21-35-07.jsonl


In [27]:
# @title Export Result to XSLX (Formatted) + Image Gen Prompt

# Example filtering (adjust as needed)
selected_model = "openai/o4-mini" # Or your best model

selected_descriptions_df = results_df[
    (results_df['Model'] == selected_model) &
    (results_df['Status'] == 'Success')
].copy() # Filter for the best model's successful runs

# Select relevant columns (don't need tokens/cost here)
columns_to_export = [
    "Image ID", "Image Path", "Category", "Model", # Keep original model info
    "primary_subject", "composition_framing", "background_environment",
    "lighting_color", "texture_materials", "text_branding",
    "mood_atmosphere", "overall_style"
]
# Ensure only existing columns are selected
columns_to_export = [col for col in columns_to_export if col in selected_descriptions_df.columns]
export_df = selected_descriptions_df[columns_to_export]

def create_image_prompt(row):
    """Creates a formatted string prompt from row data."""
    prompt_parts = []
    # Define column names and their desired labels in the prompt string
    # Adjust this dictionary based on exactly which fields you want
    fields_to_include = {
        "Category": "Category",
        "primary_subject": "Primary subject",
        "composition_framing": "Composition framing",
        "background_environment": "Background environment",
        "lighting_color": "Lighting color",
        "texture_materials": "Texture materials",
        "text_branding": "Text branding",
        "mood_atmosphere": "Mood atmosphere",
        "overall_style": "Overall style"
    }

    for col_name, label in fields_to_include.items():
        value = row.get(col_name) # Use .get() in case a column is missing
        # Check if value is not null/NaN AND not an empty string after stripping whitespace
        if pd.notna(value) and str(value).strip():
            prompt_parts.append(f"{label}: {str(value).strip()}") # Add label and value

    return ", ".join(prompt_parts) # Join valid parts with ", "

# Apply the function row-wise AFTER filtering but BEFORE splitting/formatting for Excel
# Ensure export_df exists and potentially has rows before applying
if not export_df.empty:
     export_df['image_prompt'] = export_df.apply(create_image_prompt, axis=1)
else:
     # If df is empty, add an empty column so subsequent code doesn't fail if it expects it
     export_df['image_prompt'] = pd.Series(dtype='object')

# --- Configuration for Excel Output ---

# Define columns to actually display in the table on each sheet (EXCLUDING Category & Model)
columns_to_display_in_table = [
    "Image ID", "Image Path", "primary_subject", "composition_framing",
    "background_environment", "lighting_color", "texture_materials",
    "text_branding", "mood_atmosphere", "overall_style", "image_prompt"
]
# Ensure these columns actually exist in the dataframe
columns_to_display_in_table = [col for col in columns_to_display_in_table if col in export_df.columns]


# Define suggested widths ONLY for the columns being displayed in the table
# Adjust widths as needed to help fit content on one page with wrapping
col_widths = {
    'Image ID': 8,
    'Image Path': 10,
    'primary_subject': 25,
    'composition_framing': 25,
    'background_environment': 25,
    'lighting_color': 25,
    'texture_materials': 25,
    'text_branding': 25,
    'mood_atmosphere': 25,
    'overall_style': 25,
    'image_prompt': 25,
}
# Filter widths dictionary to only include columns we are actually displaying
col_widths = {k: v for k, v in col_widths.items() if k in columns_to_display_in_table}

# Define which of the DISPLAYED columns should have text wrapping enabled
columns_to_wrap = [col for col in columns_to_display_in_table if col not in ['Image ID']]


# --- Prepare for Excel Export ---
now = datetime.datetime.now()
# Use underscores in timestamp for filename safety
formatted_date_time = now.strftime("%Y-%m-%d_%H-%M-%S")
# CHANGE FILENAME to reflect Excel output and structure
# Replace invalid filename characters from model name
safe_model_name = re.sub(r'[\\/*?:"<>|]', '_', selected_model)
export_filename_excel = f"image_descriptions_by_category_{formatted_date_time}.xlsx"

# Function to sanitize sheet names (Excel limits length and characters)
def sanitize_sheet_name(name):
    name = str(name) # Ensure it's a string
    name = re.sub(r'[\\/*?:\[\]]', '_', name) # Remove invalid characters
    return name[:31] # Truncate to Excel's limit

# Get unique categories and the model name from the filtered dataframe
if not export_df.empty:
    unique_categories = sorted(export_df['Category'].unique())
    # Model name should be consistent due to the initial filter
    model_name = export_df['Model'].iloc[0]
else:
    unique_categories = []
    model_name = selected_model # Fallback if df is empty but filter was applied
    print(f"Warning: Filtered DataFrame 'export_df' for model '{selected_model}' is empty.")


# --- Create Excel File with Sheets per Category (Replaces CSV Export and Truncation)---
# REMOVED: Truncation logic (path_max_length, paragraph_max_length, loop for truncation)
# REMOVED: export_df_truncated definition

with pd.ExcelWriter(export_filename_excel, engine='xlsxwriter') as writer:
    if not unique_categories:
        # Handle case of no data/categories after filtering
        workbook = writer.book
        worksheet = workbook.add_worksheet("No Data")
        worksheet.write('A1', f"No successful runs found for model '{selected_model}'.")
        print(f"INFO: No data to write for model '{selected_model}'. Excel file created with 'No Data' sheet.")
    else:
        print(f"Processing {len(unique_categories)} categories for model '{model_name}'...")
        for category in unique_categories:
            sheet_name = sanitize_sheet_name(category)
            print(f"  Creating sheet: {sheet_name} (Category: {category})")

            # Filter data for the current category
            df_category_subset = export_df[export_df['Category'] == category].copy()

            # Select only the columns we want to display in the table for this sheet
            df_to_write = df_category_subset[columns_to_display_in_table]

            # Write the filtered data to the specific sheet, starting data below headers
            # startrow=2 means data headers begin on Excel row 3
            df_to_write.to_excel(writer, sheet_name=sheet_name, index=False, startrow=2, header=True)

            # --- Get workbook and worksheet objects ---
            workbook = writer.book
            worksheet = writer.sheets[sheet_name]

            # --- Write Custom Header (Category & Model) in Row 1 (index 0) ---
            header_format = workbook.add_format({
                'bold': True, 'font_size': 11, 'align': 'left', 'valign': 'vcenter',
                'bg_color': '#F0F0F0', 'bottom': 1 # Light grey bg, bottom border
            })
            header_text = f"Category: {category}   |   Model: {model_name}"
            worksheet.merge_range('A1:C1', header_text, header_format) # Merge A1:C1 for title
            worksheet.set_row(0, 20) # Set height for the header row (row 1 in Excel)

            # --- Apply Formatting (Wrapping & Widths) to Data Table Columns ---
            # Format for cells where text should wrap
            wrap_format = workbook.add_format({
                'text_wrap': True,
                'valign': 'top' # Align text to the top in wrapped cells
            })
            # Optional: Format for cells that don't need wrapping (e.g., Image ID)
            # default_format = workbook.add_format({'valign': 'top'})

            # Iterate through columns that are actually written to the sheet
            for i, col_name in enumerate(df_to_write.columns):
                width = col_widths.get(col_name, 15) # Get width, default 15 if not set
                if col_name in columns_to_wrap:
                    # Apply width AND the wrap format to this column
                    worksheet.set_column(i, i, width, wrap_format)
                else:
                    # Apply width only (using default cell format)
                    worksheet.set_column(i, i, width) #, default_format)

            # Optional: Freeze top header row and column headers row for scrolling
            # Freezes rows 1, 2, 3 (our header, blank row, column names)
            worksheet.freeze_panes(3, 0)

# Use the new Excel filename in the final print statement
print(f"\n✅ Descriptions exported by category to {export_filename_excel}")

Processing 1 categories for model 'qwen/qwen2.5-vl-32b-instruct'...
  Creating sheet: Menu Displays (Category: Menu Displays)

✅ Descriptions exported by category to image_descriptions_by_category_2025-04-23_22-25-54.xlsx


<ipython-input-27-c0ee2fc300df>:50: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  export_df['image_prompt'] = export_df.apply(create_image_prompt, axis=1)


In [16]:
# @title Generate HTML Report for Visual Evaluation

# --- Configuration for HTML Report ---
HTML_REPORT_FILENAME = "fnb_evaluation_report_5.html"
# Set this path if your images are in a specific folder relative to the notebook
# or use absolute paths in IMAGES_TO_PROCESS. Leave as "" if paths in
# IMAGES_TO_PROCESS are already correct relative to the HTML file location.
# IMAGE_BASE_PATH_FOR_HTML = "images/" # Example: "images/" or "/path/to/images/"
IMAGE_BASE_PATH_FOR_HTML = "" # Assume paths in df are correct relative to HTML

# Define the HTML template using Jinja2 syntax in a string
html_template_string = """
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>F&B Image Evaluation Report</title>
    <style>
        body { font-family: sans-serif; margin: 20px; line-height: 1.6; }
        .image-section { margin-bottom: 40px; border-bottom: 2px solid #eee; padding-bottom: 20px; }
        .image-container img { max-width: 400px; max-height: 400px; border: 1px solid #ccc; margin-bottom: 15px; }
        table { border-collapse: collapse; width: 100%; margin-top: 15px; font-size: 0.9em; }
        th, td { border: 1px solid #ddd; padding: 8px; text-align: left; vertical-align: top; }
        th { background-color: #f2f2f2; }
        tr:nth-child(even) { background-color: #f9f9f9; }
        td:first-child { font-weight: bold; min-width: 150px; } /* Model name column */
        .status-success { color: green; }
        .status-error { color: red; font-weight: bold; }
        .description-cell { white-space: pre-wrap; word-wrap: break-word; } /* Wrap long text */
    </style>
</head>
<body>
    <h1>F&B Image Evaluation Report</h1>
    <p>Date Generated: {{ generation_date }}</p>

    {% for image_id, group in results.groupby('Image ID') %}
    <div class="image-section">
        <h2>Image ID: {{ image_id }}</h2>
        <p><strong>Category:</strong> {{ group['Category'].iloc[0] }}</p>
        <div class="image-container">
            {% set img_path = image_base_path + group['Image Path'].iloc[0] %}
            <img src="{{ img_path }}" alt="Image {{ image_id }}" onerror="this.alt='Image not found at {{ img_path }}'; this.style.border='1px solid red';">
        </div>

        <table>
            <thead>
                <tr>
                    <th>Model</th>
                    <th>Status</th>
                    <th>Total Tokens</th>
                    <th>Est. Cost (USD)</th>
                    <th>Primary Subject</th>
                    <th>Composition/Framing</th>
                    <th>Background/Environment</th>
                    <th>Lighting/Color</th>
                    <th>Texture/Materials</th>
                    <th>Text/Branding</th>
                    <th>Mood/Atmosphere</th>
                    <th>Overall Style</th>
                </tr>
            </thead>
            <tbody>
                {% for index, row in group.iterrows() %}
                <tr>
                    <td>{{ row['Model'] }}</td>
                    <td class="status-{{ row['Status'].lower() }}">{{ row['Status'] }}</td>
                    <td>{{ row['Total Tokens'] }}</td>
                    <td>{{ "%.6f"|format(row['Estimated Cost (USD)']) }}</td>
                    <td class="description-cell">{{ escape(row['primary_subject']) }}</td>
                    <td class="description-cell">{{ escape(row['composition_framing']) }}</td>
                    <td class="description-cell">{{ escape(row['background_environment']) }}</td>
                    <td class="description-cell">{{ escape(row['lighting_color']) }}</td>
                    <td class="description-cell">{{ escape(row['texture_materials']) }}</td>
                    <td class="description-cell">{{ escape(row['text_branding']) }}</td>
                    <td class="description-cell">{{ escape(row['mood_atmosphere']) }}</td>
                    <td class="description-cell">{{ escape(row['overall_style']) }}</td>
                </tr>
                {% endfor %}
            </tbody>
        </table>
    </div>
    {% endfor %}

</body>
</html>
"""

def generate_html_report(df, output_filename, image_base_path=""):
    """Generates an HTML report from the results DataFrame."""
    if df.empty:
        print("❌ Cannot generate report: Results DataFrame is empty.")
        return

    # Ensure necessary columns exist, especially 'Image Path'
    if 'Image Path' not in df.columns:
        print("❌ Cannot generate report: 'Image Path' column missing in results.")
        return

    try:
        # Using Jinja2 directly with the template string
        env = Environment(loader=None, autoescape=select_autoescape(['html', 'xml']))
        env.globals['escape'] = html.escape # Add escape function to template context
        template = env.from_string(html_template_string)

        generation_date = time.strftime("%Y-%m-%d %H:%M:%S %Z")

        # Render the template
        html_content = template.render(
            results=df,
            generation_date=generation_date,
            image_base_path=image_base_path
        )

        # Write to file
        with open(output_filename, "w", encoding="utf-8") as f:
            f.write(html_content)
        print(f"✅ HTML report generated successfully: {output_filename}")

    except Exception as e:
        print(f"❌ Error generating HTML report: {e}")
        traceback.print_exc()

# --- Generate the Report ---
# Make sure results_df exists and is populated from the previous cell
if 'results_df' in locals() and not results_df.empty:
     generate_html_report(results_df, HTML_REPORT_FILENAME, IMAGE_BASE_PATH_FOR_HTML)
elif not results_list:
     print("⚠️ Skipping HTML report generation because no results were generated in the workflow.")
else:
     print("⚠️ Skipping HTML report generation. Ensure the main workflow cell has been run successfully and 'results_df' exists.")


from IPython.display import HTML

with open('fnb_evaluation_report_5.html', 'r') as f:  # Assuming your file is named 'fnb_evaluation_report.html'
  html_content = f.read()

display(HTML(html_content))


✅ HTML report generated successfully: fnb_evaluation_report_5.html


Model,Status,Total Tokens,Est. Cost (USD),Primary Subject,Composition/Framing,Background/Environment,Lighting/Color,Texture/Materials,Text/Branding,Mood/Atmosphere,Overall Style
gemini-2.5-pro-exp-03-25,Success,3356,0.000000,"Two sandwiches featured prominently. The top sandwich is a &#x27;Cheesy Smoked Beef Sandwich&#x27; (芝芝熏牛三明治) served in a golden-brown, soft brioche-style bun, filled with layers of pink sliced smoked beef (pastrami), sautéed sliced mushrooms, white onions, and possibly melted cheese. The bottom sandwich is a &#x27;Musang King Durian Tender Chicken Croissant&#x27; (猫山王榴莲嫩鸡牛角), a golden-brown, flaky croissant filled with fluffy yellow scrambled eggs, pieces of tender chicken, and a creamy sauce (likely the durian salad dressing), with some herbs sprinkled on the eggs.","Overhead shot (top-down view). The two sandwiches are arranged diagonally across the frame, appearing to burst through the background paper. The composition is dynamic but relatively balanced. Framing is a medium shot, capturing the sandwiches and surrounding text elements clearly.","The background is a flat, light brown, textured paper surface resembling parchment or a craft paper bag. The sandwiches are presented as if tearing through ripped holes in this paper, with torn edges visible around them. A few loose, cooked mushroom slices are scattered near the top sandwich as foreground/midground elements. The depth of field is relatively shallow, keeping the sandwiches and immediate ripped paper edges sharp, while the flat background paper is slightly softer.","Bright, even studio lighting, likely positioned slightly above and in front of the subjects. Shadows are soft and minimal, primarily visible under the sandwiches and creating depth around the torn paper edges. The dominant color palette includes warm browns (bread, paper), golden yellow (croissant, eggs, cheese), pink (beef), white (onions, cheese), accented by blue and orange text elements. The overall color temperature is warm and inviting.","Visible textures include the glossy, golden crust of the brioche bun; the flaky, layered texture of the croissant; the soft, slightly wrinkled surface of the sliced beef; the smooth, possibly melted texture of cheese; the fluffy, moist appearance of the scrambled eggs; the fibrous texture of the cooked mushrooms; and the matte, slightly crumpled texture of the brown paper background. The text elements have a distressed, stamped texture.","Extensive text overlay typical of a promotional poster. Top features large, distressed blue block text &quot;买一送一&quot; (Buy One Get One Free) and large orange distressed text &quot;灵感简餐&quot; (Inspired Light Meal). Specific item names (&quot;芝芝熏牛三明治&quot;, &quot;猫山王榴莲嫩鸡牛角&quot;) and descriptions/ingredients are placed in distressed blue boxes near each sandwich using a mix of vertical and horizontal layouts. Promotional details (&quot;10.26-10.28, 喜茶GO小程序下单&quot;, &quot;买一送一&quot;, fine print) are at the bottom in smaller black sans-serif font. A price (&quot;¥25/个&quot;) is highlighted in a red-orange starburst shape. A small blue icon of a person drinking appears near the top right. The overall text style is bold, distressed, and promotional.","Energetic, appetizing, and promotional. The &#x27;bursting through paper&#x27; visual creates a dynamic and slightly rustic feel, while the food photography aims for deliciousness. It feels casual yet enticing.","Photorealistic food presentation combined with graphic design poster elements. The style uses clean studio lighting and a shallow depth of field to emphasize the food. The background and text employ a distressed, slightly rustic graphic style. Post-processing likely includes color enhancement to make the food look appealing and saturation adjustments to balance the food and graphic elements."
